In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import ast
from collections import Counter
import scipy.stats as stats

## Data preprocessing and aggregation

In [ ]:
tqdm.pandas()

In [ ]:
df_elsi = pd.read_csv('4-elsi-21-22.csv', skiprows=6, dtype={'Agency ID - NCES Assigned [District] Latest available year': str})
df_elsi['nces_id'] = df_elsi['Agency ID - NCES Assigned [District] Latest available year']

In [ ]:
# Filter district types
mask = df_elsi['Agency Type [District] 2021-22'].map(lambda s: False if pd.isna(s) else s[0] in ['1', '2', '7'])
df_elsi = df_elsi[mask].copy()

In [ ]:
# Run dl check for fb posts first
df_accounts_3 = pd.read_csv('all-dl-coding-control.csv')

In [ ]:
# Robustness check estimation
df_elsi['hasFB'] = df_elsi.nces_id.map(lambda s: s in df_accounts_3['nces_id'].values)

In [ ]:
# Run webscraping first
df_web = pd.read_csv('1-scraped-data-final-schools.csv', dtype={'nces_id': str})

df_web['nces_id'] = df_web['nces_id'].map(lambda s: s.replace('.0', ''))

In [ ]:
df_elsi['hasWEB'] = df_elsi.nces_id.map(lambda s: s in df_web['nces_id'].values)

In [ ]:
def add_race_ratio(df_elsi, input_col, output_col):
    numerator = df_elsi[input_col].map(lambda x: int(x) if isinstance(x, str) and x.isdigit() else pd.NA)
    denominator = df_elsi['Total Students All Grades (Excludes AE) [District] 2021-22'].map(lambda x: int(x) if isinstance(x, str) and x.isdigit() else pd.NA)
    denominator = denominator.map(lambda x: pd.NA if pd.isna(x) else x if x!=0 else pd.NA)
    df_elsi[output_col] = numerator/denominator
    df_elsi['n_students'] = df_elsi['Total Students All Grades (Excludes AE) [District] 2021-22']
    df_elsi['nces_id'] = df_elsi['Agency ID - NCES Assigned [District] Latest available year']
    return df_elsi

df_elsi = add_race_ratio(df_elsi, 'White Students [District] 2021-22', 'ratio_white')

df_elsi = add_race_ratio(df_elsi, 'American Indian/Alaska Native Students [District] 2021-22', 'ratio_asian')
df_elsi = add_race_ratio(df_elsi, 'Hispanic Students [District] 2021-22', 'ratio_hispanic')
df_elsi = add_race_ratio(df_elsi, 'Black or African American Students [District] 2021-22', 'ratio_black')
df_elsi = add_race_ratio(df_elsi, 'Nat. Hawaiian or Other Pacific Isl. Students [District] 2021-22', 'ratio_pac_is')
df_elsi = add_race_ratio(df_elsi, 'Two or More Races Students [District] 2021-22', 'ratio_multirace')

In [ ]:
# Join poverty and english language index
join_this1 = pd.read_csv('crdc_lep_sum.csv', dtype={'leaid': str})[['leaid', 'ratio_lep']]
join_this1.rename(columns={'leaid': 'nces_id'}, inplace=True)

join_this2 = pd.read_csv('ccd_frpl.csv', dtype={'LEAID': str})[['LEAID', 'ratio_frpl']]
join_this2.rename(columns={'LEAID': 'nces_id'}, inplace=True)

df_join = df_elsi[['nces_id', 'noFB', 'hasWEB', 'ratio_white', 'ratio_black', 'ratio_hispanic', 'ratio_asian',
         'ratio_pac_is', 'ratio_multirace', 'n_students']]

df_join = df_join.merge(join_this1, how='left', on='nces_id').merge(join_this2, how='left', on='nces_id')

In [ ]:
df_join['n_students'] = df_join['n_students'].map(lambda x: int(x) if isinstance(x, str) and x.isdigit() else pd.NA)

## Descriptive analysis

In [ ]:
grouped_df = df_join.groupby(['hasFB'])[['ratio_white', 'ratio_black', 'ratio_hispanic', 'ratio_asian',
         'ratio_pac_is', 'ratio_multirace', 'n_students']].mean().reset_index()

grouped_df

In [ ]:
grouped_df = df_join.groupby(['hasFB'])[['ratio_white', 'ratio_black', 'ratio_hispanic', 'ratio_asian',
         'ratio_pac_is', 'ratio_multirace', 'n_students', 'ratio_frpl', 'ratio_lep']].mean().reset_index()


grouped_df

In [ ]:
grouped_df = df_join.groupby(['hasWEB'])[['ratio_white', 'ratio_black', 'ratio_hispanic', 'ratio_asian',
         'ratio_pac_is', 'ratio_multirace', 'n_students']].mean().reset_index()

grouped_df

In [ ]:
grouped_df = df_join.groupby(['hasWEB'])[['ratio_white', 'ratio_black', 'ratio_hispanic', 'ratio_asian',
         'ratio_pac_is', 'ratio_multirace', 'n_students', 'ratio_frpl', 'ratio_lep']].mean().reset_index()


grouped_df

In [ ]:
grouped_df = df_join.groupby(['hasFB', 'hasWEB'])[['ratio_white', 'ratio_black', 'ratio_hispanic', 'ratio_asian',
         'ratio_pac_is', 'ratio_multirace', 'n_students']].mean().reset_index()


grouped_df

In [ ]:
df_join.groupby(['hasFB', 'hasWEB']).size()

In [ ]:
df_join_check = df_join.copy()

In [ ]:
grouped_df = df_join.groupby(['hasFB', 'hasWEB'])[['ratio_white', 'ratio_black', 'ratio_hispanic', 'ratio_asian',
         'ratio_pac_is', 'ratio_multirace', 'n_students', 'ratio_frpl', 'ratio_lep']].mean().reset_index()


grouped_df

In [ ]:
# Overall
df_join[['ratio_white', 'ratio_black', 'ratio_hispanic', 'ratio_asian',
         'ratio_pac_is', 'ratio_multirace', 'n_students', 'ratio_frpl', 'ratio_lep']].mean()

In [ ]:
numerator = df_elsi['White Students [District] 2021-22'].map(lambda x: int(x) if isinstance(x, str) and x.isdigit() else pd.NA)
denominator = df_elsi['Total Students All Grades (Excludes AE) [District] 2021-22'].map(lambda x: int(x) if isinstance(x, str) and x.isdigit() else pd.NA)
denominator = denominator.map(lambda x: pd.NA if pd.isna(x) else x if x!=0 else pd.NA)
df_elsi['ratio_white'] = numerator/denominator
df_elsi['nces_id'] = df_elsi['Agency ID - NCES Assigned [District] Latest available year']

In [ ]:
df_elsi.groupby('noFB')['ratio_white'].mean()

In [ ]:
df_elsi.groupby('noWEB')['ratio_white'].mean()

# Statistical testing

In [ ]:
df_matrix = df_accounts_3.merge(df_web, how='outer', on='nces_id').drop_duplicates(subset=['nces_id'])

grouped_df = df_matrix.groupby(['dl_fb', 'dl_web']).size().reset_index(name='count')
pivot_df = grouped_df.pivot(index='dl_fb', columns='dl_web', values='count')
print(pivot_df)

# Perform a chi-squared test
chi2_stat, p_value, dof, _ = stats.chi2_contingency(pivot_df)

print("Chi-squared statistic:", chi2_stat)
print("P-value:", p_value)
print("Degrees of Freedom:", dof)